# 深入理解Transformer LLM

In [1]:
import os
os.environ["HF_HOME"] = "/openbayes/home/huggingface"

In [1]:
import transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# 从hugging face模型库下载并加载与模型匹配的tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto', # 将模型自动匹配到可用的设备上
    torch_dtype = "auto", # 自动选择最优的数据类型以节省显存并加速推理
    trust_remote_code = True # 允许加载远程仓库中自定义的模型代码
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [3]:
generator = pipeline(
    "text-generation",# 指定任务类型为文本生成
    model = model, # 使用加载的预训练模型对象
    tokenizer = tokenizer, # 使用与模型匹配的分词器
    return_full_text = False, # 控制返回的结果只包含新生成的内容
    max_new_tokens = 50, # 最大新token数
    do_sample = False, # 设置是否随机采样
)

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
prompt = "《梦游天姥吟留别》的作者是谁"
output = generator(prompt)

print(output)
print("++++++++++++++++")
print(output[0]['generated_text'])

[{'generated_text': '？（ ） A. 李白 B. 王维 C. 杜甫 D. 白居易\n\n李白\n\n“我歌月徘徊，我舞影零乱”出自哪首诗？（ ） A'}]
++++++++++++++++
？（ ） A. 李白 B. 王维 C. 杜甫 D. 白居易

李白

“我歌月徘徊，我舞影零乱”出自哪首诗？（ ） A


In [5]:
message = [
    {"role":"system","content": "你是一个很有用的助手"},
    {"role":"user","content":prompt}
]

text = tokenizer.apply_chat_template(
    message,
    tokenize = False, # 表示只返回字符串形式，不进行token编码
    add_generation_prompt = True # 如果不加 add_generation_prompt=True，模型可能不会开始生成 assistant 回复。
)

print(text)

<|im_start|>system
你是一个很有用的助手<|im_end|>
<|im_start|>user
《梦游天姥吟留别》的作者是谁<|im_end|>
<|im_start|>assistant



In [6]:
generator(text)[0]["generated_text"]

'李白'

## 输入格式：纯文本 vs 聊天模板（Prompt 形式）
| 比较点                        | 纯文本 prompt    | Chat Template prompt      |
| -------------------------- | ------------- | ------------------------- |
| 输入格式                       | 用户直接写一句话      | system/user/assistant 结构化 |
| 模型模式                       | 更像语言补全        | 对话问答模式                    |
| 是否遵循 system 指令             | 视模型情况而定（通常较弱） | 强烈遵守                      |
| 输出风格                       | 可能短、断句、补全式    | 更像聊天助手                    |
| 是否需要 add_generation_prompt | 否             | 是（对话模型）                   |



In [7]:
# model output 可以怎么用
prompt = "The captial of France is"

input_ids = tokenizer(prompt,return_tensors='pt').input_ids.to(model.device)

# model.model返回什么
# model.model是其transformers主体，返回一个BaseModelOutputWithPast或类似对象，里面包含last_hidden_state，表示模型对每个token的向量表示
model_output = model.model(input_ids)
print(model_output[0].shape)

# model.lm_head返回什么
#lm_head是一个线性投影层，把隐藏层输出映射到词表大小的维度，用于生成每个token的预算分布
lm_head_output = model.lm_head(model_output[0])
print(lm_head_output.shape)

torch.Size([1, 6, 896])
torch.Size([1, 6, 151936])


In [8]:
# model.lm_head的使用
# 获取最后一个token的预测分布
last_token_logits = lm_head_output[0,-1]
print(last_token_logits.shape)
# 获取概率最大的词
predicted_token_id = last_token_logits.argmax().item()
# 解码为词
predicted_word = tokenizer.decode(predicted_token_id)
print(predicted_word)

torch.Size([151936])
 Paris


In [9]:
# model.generate的使用
output = model.generate(input_ids)

print(tokenizer.decode(output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The captial of France is Paris, which has a population of 2.3 million people and an area of 15


| 方法                    | 用途                               | 输出             | 是否推荐用于生成    |
| --------------------- | -------------------------------- | -------------- | ----------- |
| `model.generate(...)` | 自动文本生成                           | token ids      | ✅ 推荐        |
| `model.model(...)`    | 得到隐藏状态表示（不含输出头）                  | hidden states  | ❌ 不推荐直接用于生成 |
| `model.lm_head(...)`  | 把 hidden states 映射到 vocab logits | logits（用于生成预测） | ✅ 可用于研究内部细节 |


## 推理方式：pipeline vs model.generate（调用方式）
| 比较点                   | pipeline | model.generate |
| --------------------- | -------- | -------------- |
| 分词                    | 自动       | 手动             |
| 解码                    | 自动       | 手动             |
| 输出格式                  | 封装成 dict | 原始 tensor      |
| 易用性                   | 简单       | 需要更多代码         |
| 控制力                   | 中等       | 强              |
| 是否可与 chat template 配合 | ✔ 可以     | ✔ 可以           |


In [51]:
#  KV cache 的原理，以及在推理的时候怎么使用？

prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

In [58]:
%%timeit -n 1
# %%timeit 是 IPython / Jupyter Notebook 的魔法命令，用于 测量代码块的运行时间。
# -n 1 表示 只运行 1 次。默认情况下，timeit 会多次运行取平均，这里你只测一次。

generation_output = model.generate(
    input_ids = input_ids,
    max_new_tokens = 1000,
    use_cache = True # 启用缓存机制，在生成多个token时，避免重复计算历史token，加快速度
)

The slowest run took 7.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1min 18s ± 55.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=1000,
  use_cache=False
)